# [05] Action-Value Iteration in Frozen Lake

### Imports & Constants

In [1]:
import gym
import collections
import os
import datetime
from torch.utils.tensorboard import SummaryWriter

ENV_NAME = "FrozenLake-v0"
#ENV_NAME = "FrozenLake8x8-v0"      # uncomment for larger version
GAMMA = 0.9
NUM_TEST_EPISODES = 20

<br> 

### Agent

`rewards`: $ rewards[(s, a, s')] = \{ c \} $

`transitions`: $ transitions[(s, a)] = \{ s_1:c_1, s_2:c_2, ... \} $

`action_values`: $ state\text{_}values[(s, a)] = \{ v \} $

In [24]:
class Agent:
    
    def __init__(self):
        
        # Create & reset environment 
        self.env = gym.make(ENV_NAME)
        self.state = self.env.reset()
        
        # Create dictionaries for rewards, transitions & state-values
        self.rewards = collections.defaultdict(float)
        self.transitions = collections.defaultdict(collections.Counter)
        self.action_values = collections.defaultdict(float)

    def play_n_random_steps(self, num_steps):
        '''Play `num_steps` steps in environment'''
        
        for _ in range(num_steps):
            
            # Sample random action & take step in environment 
            action = self.env.action_space.sample()
            new_state, reward, is_done, _ = self.env.step(action)
            
            # Update rewards & transitions
            self.rewards[(self.state, action, new_state)] = reward
            self.transitions[(self.state, action)][new_state] += 1
            
            # Update state 
            self.state = self.env.reset() if is_done else new_state

    def select_action(self, state):
        '''Select best action from `state`'''
        
        best_action, best_value = None, None
        
        for action in range(self.env.action_space.n):
            
            # Get action value from table 
            action_value = self.action_values[(state, action)]
            
            # Update best action value 
            if best_value is None or best_value < action_value:
                best_value = action_value
                best_action = action
        
        return best_action
    
    def play_episode(self, env):
        '''Play a single episode'''

        # Reset environment 
        total_reward = 0.0
        state = env.reset()
        
        while True:
            
            # Select action and take step 
            action = self.select_action(state)
            new_state, reward, is_done, _ = env.step(action)
            
            # Update rewards, transitions & total reward 
            self.rewards[(state, action, new_state)] = reward
            self.transitions[(state, action)][new_state] += 1
            total_reward += reward
            
            if is_done:
                break
            state = new_state
        
        return total_reward            
            
    def value_iteration(self):
        '''Action-Value Iteration for all states'''
        
        for state in range(self.env.observation_space.n):
            for action in range(self.env.action_space.n):
                
                # Counts of each state 
                # target_counts = {s_1:c_1, s_2:c_2, ... }
                target_counts = self.transitions[(state, action)]
                
                # Sum of counts of each state 
                total = sum(target_counts.values())
                
                action_value = 0.0
                
                # Calculate all possible action values 
                for target_state, count in target_counts.items():
                    
                    # Get reward 
                    key = (state, action, target_state)
                    reward = self.rewards[key]
                    
                    # Select best action for each target state 
                    best_action = self.select_action(target_state)
                    
                    # Calculate action value 
                    val = reward + GAMMA * self.action_values[(target_state, best_action)]
                    action_value += (count / total) * val
                
                # Assign final action value to table 
                self.action_values[(state, action)] = action_value

##### `value_iteration()` Fromula
$ Q(s, a) = r(s, a) + \gamma \text{max}_{a' \in A} Q(s', a') $

<br> 

### Main

In [23]:
# Create environment & agent 
test_env = gym.make(ENV_NAME)
agent = Agent()

# Initialize Tensorboard 
log_dir = os.path.join('./runs/05_runs', datetime.datetime.now().strftime("%Y:%m:%d-%H:%M:%S"))
writer = SummaryWriter(log_dir)

iter_no = 0
best_reward = 0.0

while True:
    iter_no += 1
    
    # Play random steps to fill `rewards` and `transitions` tables with new data 
    agent.play_n_random_steps(100)
    
    # State-value iteration over all states 
    agent.value_iteration()

    # Play `NUM_TEST_EPISODES` episodes and accumulate the reward 
    reward = 0.0
    for _ in range(NUM_TEST_EPISODES):
        reward += agent.play_episode(test_env)
    reward /= NUM_TEST_EPISODES
    writer.add_scalar("Reward", reward, iter_no)
        
    # If we have a new best reward 
    if reward > best_reward:
        print("Best reward updated %.3f -> %.3f" % (best_reward, reward))
        best_reward = reward
    
    # If we've solved the environment
    if reward > 0.80:
        print("Solved in %d iterations!" % iter_no)
        break

writer.close()

Best reward updated 0.000 -> 0.400
Best reward updated 0.400 -> 0.450
Best reward updated 0.450 -> 0.550
Best reward updated 0.550 -> 0.650
Best reward updated 0.650 -> 0.750
Best reward updated 0.750 -> 0.850
Solved in 18 iterations!


<br>

### Tensorboard

In [ ]:
!tensorboard dev upload --logdir='./runs/05_runs' --name='05_action_value_iteration_frozenlake'

<img src='./runs/05_runs/tensorboard.png'>

<br>